In [2]:
import pandas as pd
import re
import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('chinese_cars_final.csv')
data

,Название,Марка,Модель,"Пробег, км","Мощность двигателя, л","Мощность двигателя, л.с",Год выпуска,"Цена от, руб","Цена до, руб",Цвет кузова,Коробка передач,Привод,Тип топлива,Город продажи
0,Chery Tiggo 7 Pro Max I Рестайлинг,Chery,Tiggo 7 Pro Max I Рестайлинг,Новый,1.6,150,2025,2 149 000,Not available,Active AWD,робот,полный,Бензин,Балашиха
1,Haval Jolion I Рестайлинг,Haval,Jolion I Рестайлинг,Новый,1.5,150,2025,1 820 000,Not available,Elite,робот,полный,Бензин,Химки
2,Haval H9 II,Haval,H9 II,Новый,2.4,184,2024,3 454 000,Not available,Elite,автомат,полный,Дизель,Москва
3,Chery Tiggo 9 I,Chery,Tiggo 9 I,Новый,2.0,249,2024,3 535 000,Not available,Ultra,автомат,полный,Бензин,Москва
4,Haval H3,Haval,H3,12 000,1.5,177,2024,2 320 000,2 320 000,серый,робот,полный,Бензин,Москва
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3010,Geely Monjaro (Xingyue L) I Рестайлинг,Geely,Monjaro (Xingyue L) I Рестайлинг,10,2.0,238,2025,3 830 000,3 830 000,серебристый,автомат,полный,Бензин,Москва
3011,Geely Icon,Geely,Icon,1,1.5,181,2024,2 699 999,2 699 999,голубой,робот,передний,Бензин,Москва
3012,Geely Icon,Geely,Icon,10,1.5,181,2024,2 699 999,2 699 999,серый,робот,передний,Бензин,Москва
3013,Geely Atlas Pro,Geely,Atlas Pro,58 000,1.5,177,2022,2 100 000,2 100 000,синий,робот,полный,Бензин,Одинцово


# Предобработка

In [4]:
data.dtypes

Название                   object
Марка                      object
Модель                     object
Пробег, км                 object
Мощность двигателя, л      object
Мощность двигателя, л.с    object
Год выпуска                 int64
Цена от, руб               object
Цена до, руб               object
Цвет кузова                object
Коробка передач            object
Привод                     object
Тип топлива                object
Город продажи              object
dtype: object

In [5]:
data.rename(columns={'Пробег, км' : 'Пробег', 
                     'Мощность двигателя, л': 'Мощность_1', 
                     'Мощность двигателя, л.с': 'Мощность_2',
                     'Цена от, руб': 'Цена', 
                     'Цвет кузова': 'Цвет',
                     'Коробка передач': 'Коробка_передач',
                     'Тип топлива': 'Тип_топлива',
                     'Город продажи': 'Город'}, inplace=True)

In [8]:
for col in data.columns:
    print(f'Название колонки {col}')
    print(data[col].value_counts())
    print('----------------------------------------')
    print()

Название колонки Название
Название
Chery Tiggo 7 Pro Max I Рестайлинг        326
Geely Monjaro (Xingyue L) I Рестайлинг    292
Chery Tiggo 4 I Рестайлинг 2              158
Chery Tiggo 9 I                           154
Geely Atlas Pro                           112
                                         ... 
Haval H8                                    1
Chery Tiggo 5 I Рестайлинг                  1
Chery Tiggo 8 Plus I                        1
Chery M11 (A3)                              1
Geely GS I                                  1
Name: count, Length: 82, dtype: int64
----------------------------------------

Название колонки Марка
Марка
Chery    1232
Geely    1106
Haval     675
Name: count, dtype: int64
----------------------------------------

Название колонки Модель
Модель
Tiggo 7 Pro Max I Рестайлинг        326
Monjaro (Xingyue L) I Рестайлинг    292
Tiggo 4 I Рестайлинг 2              158
Tiggo 9 I                           154
Atlas Pro                           112
         

In [9]:
# Убираем строки, где мощность принимает странное значение
data = data[~data['Мощность_1'].str.contains('218')]

In [10]:
# Убираем лишние символы
#data['Mощность_2'] = data['Мощность_2'].replace('160 кВт', '160')

In [11]:
# Убираеи записи, по которым не знаем цену
data = data[data['Цена'] != 'Not available']

In [12]:
# В колонку Цвета попала комплектация, убираем странные записи и оставляем только цвета
real_colors = [
    'серый', 'белый', 'чёрный', 'серебристый', 'зелёный', 'синий', 
    'жёлтый', 'голубой', 'фиолетовый', 'красный', 'пурпурный', 
    'коричневый', 'оранжевый', 'золотистый', 'Ultra Black'
]

data = data[data['Цвет'].isin(real_colors)]

In [13]:
# Создаем новый признак с группами по цвету

color_groups = {
    'Нейтральные': ['серый', 'белый', 'чёрный', 'серебристый', 'Ultra Black'],  
    'Холодные': ['синий', 'голубой', 'зелёный'],  
    'Тёплые': ['красный', 'жёлтый', 'оранжевый', 'золотистый'],  
    'Экзотические': ['фиолетовый', 'пурпурный', 'коричневый']  
}
def map_color_to_group(color):
    for group, colors in color_groups.items():
        if color in colors:
            return group
    return 'Другие'

data['Цвет_группа'] = data['Цвет'].apply(map_color_to_group)

In [14]:
data['Цвет_группа'].value_counts()

Цвет_группа
Нейтральные     987
Холодные        226
Тёплые           73
Экзотические     29
Name: count, dtype: int64

In [15]:
# создаем признак is_new
data['is_new'] = data['Пробег'] == 'Новый'

# Заменяем "новый" в колонке пробег на 0
data['Пробег'] = data['Пробег'].replace('Новый', '0')

In [16]:
# Определяем группы городов
city_groups = {
    'Москва': [
        'Москва',
        'Северо-Западный административный округ',
        'Братеево',
        'Тёплый Стан',
        'Коммунарка'
    ],
    'Московская область': [
        'Балашиха', 'Химки', 'Королёв', 'Мытищи', 'Подольск', 
        'Люберцы', 'Красногорск', 'Одинцово', 'Видное', 'Лобня',
        'Котельники', 'Дмитров', 'Реутов', 'Щёлково', 'Серпухов',
        'Электросталь', 'Ногинск', 'Раменское', 'Звенигород',
        'Павловский Посад', 'Орехово-Зуево', 'Можайск (100 км от Москвы)'
    ],
    'Ближние регионы (до 200 км)': [
        'Тула (170 км от Москвы)', 'Калуга (160 км от Москвы)',
        'Рязань (180 км от Москвы)', 'Владимир (170 км от Москвы)',
        'Тверь (160 км от Москвы)', 'Коломна (100 км от Москвы)',
        'Волоколамск (100 км от Москвы)', 'Суздальский район (180 км от Москвы)',
        'Асерхово (160 км от Москвы)', 'Протвино (100 км от Москвы)'
    ],
    'Города-спутники': [
        'Тимохово', 'Ржавки', 'Московский', 'Вербилки',
        'Исаково', 'Обнинск', 'Лосино-Петровский', 'Раздоры'
    ]
}

def map_city_to_group(city):
    normalized_city = city.split(' (')[0].strip()
    
    for group, cities in city_groups.items():
        if any(normalized_city in c for c in cities):
            return group
    return 'Прочие города'

data['Город_группа'] = data['Город'].apply(map_city_to_group)
data

,Название,Марка,Модель,Пробег,Мощность_1,Мощность_2,Год выпуска,Цена,"Цена до, руб",Цвет,Коробка_передач,Привод,Тип_топлива,Город,Цвет_группа,is_new,Город_группа
4,Haval H3,Haval,H3,12 000,1.5,177,2024,2 320 000,2 320 000,серый,робот,полный,Бензин,Москва,Нейтральные,False,Москва
32,Chery Tiggo 4 Pro I,Chery,Tiggo 4 Pro I,52 460,1.5,113,2024,1 490 000,1 490 000,серый,механика,передний,Бензин,Москва,Нейтральные,False,Москва
35,Haval H3,Haval,H3,6 308,1.5,177,2024,2 650 000,2 650 000,чёрный,робот,полный,Бензин,Москва,Нейтральные,False,Москва
57,Geely Okavango I Рестайлинг,Geely,Okavango I Рестайлинг,17 169,2.0,200,2023,2 980 000,2 980 000,синий,робот,передний,Бензин,Балашиха,Холодные,False,Московская область
60,Chery Tiggo 8 I Рестайлинг,Chery,Tiggo 8 I Рестайлинг,22 000,1.5,147,2022,1 500 000,1 500 000,серый,робот,передний,Бензин,Москва,Нейтральные,False,Москва
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3010,Geely Monjaro (Xingyue L) I Рестайлинг,Geely,Monjaro (Xingyue L) I Рестайлинг,10,2.0,238,2025,3 830 000,3 830 000,серебристый,автомат,полный,Бензин,Москва,Нейтральные,False,Москва
3011,Geely Icon,Geely,Icon,1,1.5,181,2024,2 699 999,2 699 999,голубой,робот,передний,Бензин,Москва,Холодные,False,Москва
3012,Geely Icon,Geely,Icon,10,1.5,181,2024,2 699 999,2 699 999,серый,робот,передний,Бензин,Москва,Нейтральные,False,Москва
3013,Geely Atlas Pro,Geely,Atlas Pro,58 000,1.5,177,2022,2 100 000,2 100 000,синий,робот,полный,Бензин,Одинцово,Холодные,False,Московская область


In [17]:
data['Город_группа'].value_counts()

Город_группа
Москва                         1044
Города-спутники                 147
Московская область               89
Ближние регионы (до 200 км)      22
Прочие города                    13
Name: count, dtype: int64

In [18]:
data['Модель'].unique()

array(['H3', 'Tiggo 4 Pro I', 'Okavango I Рестайлинг',
       'Tiggo 8 I Рестайлинг', 'Tiggo 8 Pro Max I', 'Tiggo 7 Pro Max I',
       'H9 II', 'Atlas II', 'F7x I', 'F7 I Рестайлинг',
       'Monjaro (Xingyue L) I Рестайлинг', 'Tiggo 7 Pro I',
       'Tiggo 4 I Рестайлинг', 'Jolion I', 'F7 II', 'Monjaro I',
       'Tiggo (T11) I', 'Tiggo 8 Plus I', 'Tiggo 7 Pro Max I Рестайлинг',
       'H6 Coupe I', 'Tiggo 7 Pro Plug-in Hybrid', 'Atlas Pro', 'Atlas I',
       'Arrizo 8', 'Dargo', 'Preface I (China Market) Рестайлинг',
       'Cityray', 'H5 II', 'Tiggo 8 Pro e+', 'Tiggo 8 Pro I',
       'Tugella I Рестайлинг', 'Xingyue I', 'Yuancheng (Farizon) FX',
       'Tiggo 4 Pro I Рестайлинг', 'QQ6 (S21)', 'Coolray I',
       'Coolray I Рестайлинг', 'F7 I', 'Preface I', 'Amulet (A15) I',
       'Preface I (China Market)', 'M11 (A3)', 'H9 I Рестайлинг',
       'Coolray (Binyue Cool) I Рестайлинг', 'H2',
       'Monjaro (Xingyue L) Hi-F I', 'Emgrand II', 'F7x I Рестайлинг',
       'Jolion I Рестайл

In [19]:
def extract_car_features(df):
    # 1. Базовое название модели
    df['base_model'] = df['Модель'].apply(
        lambda x: re.sub(r'\s(I+|II|III|IV|Рестайлинг|Pro|Max|Plus|Plug-in|Hybrid|China Market|\(.*?\)|\d)', '', x).strip()
    )
    
    # 2. Поколение модели (римские цифры)
    df['generation'] = data['Модель'].str.findall(r'\b(I{1,4}|IV)\b').apply(
        lambda x: x[-1] if len(x) > 0 else 'I').replace({'I':1, 'II':2, 'III':3, 'IV':4})
    
    # 3. Технологические особенности
    df['is_restyling'] = df['Модель'].str.contains('Рестайлинг').astype(int)
    df['is_pro'] = df['Модель'].str.contains('Pro').astype(int)
    df['is_max'] = df['Модель'].str.contains('Max').astype(int)
    
    # 4. Класс автомобиля (на основе экспертных правил)
    def get_car_class(row):
        brand = row['Марка']
        model = row['base_model']
        
        class_rules = {
            'Haval': {
                'H[3-9]': 'SUV',
                'Jolion|Dargo': 'Crossover',
                'F[7-9]': 'Luxury SUV'
            },
            'Chery': {
                'Tiggo [4-8]': 'SUV',
                'Arrizo|Amulet': 'Sedan',
                'QQ': 'City Car'
            },
            'Geely': {
                'Atlas|Xingyue': 'SUV',
                'Coolray|Monjaro': 'Crossover',
                'Emgrand|Preface': 'Sedan'
            }
        }
        
        for pattern, cls in class_rules[brand].items():
            if re.search(pattern, model):
                return cls
        return 'Other'
    
    df['car_class'] = df.apply(get_car_class, axis=1)
    
    
    # 5. Рыночная сегментация
    df['is_premium'] = df['base_model'].str.contains('Max|Pro|Plus|Monjaro|Xingyue').astype(int)
    
    # 6. Популярность модели
    model_popularity = df['base_model'].value_counts().to_dict()
    df['model_popularity'] = df['base_model'].map(model_popularity)
    
    return df

extract_car_features(data)

,Название,Марка,Модель,Пробег,Мощность_1,Мощность_2,Год выпуска,Цена,"Цена до, руб",Цвет,...,is_new,Город_группа,base_model,generation,is_restyling,is_pro,is_max,car_class,is_premium,model_popularity
4,Haval H3,Haval,H3,12 000,1.5,177,2024,2 320 000,2 320 000,серый,...,False,Москва,H3,1,0,0,0,SUV,0,4
32,Chery Tiggo 4 Pro I,Chery,Tiggo 4 Pro I,52 460,1.5,113,2024,1 490 000,1 490 000,серый,...,False,Москва,Tiggo,1,0,1,0,Other,0,270
35,Haval H3,Haval,H3,6 308,1.5,177,2024,2 650 000,2 650 000,чёрный,...,False,Москва,H3,1,0,0,0,SUV,0,4
57,Geely Okavango I Рестайлинг,Geely,Okavango I Рестайлинг,17 169,2.0,200,2023,2 980 000,2 980 000,синий,...,False,Московская область,Okavango,1,1,0,0,Other,0,7
60,Chery Tiggo 8 I Рестайлинг,Chery,Tiggo 8 I Рестайлинг,22 000,1.5,147,2022,1 500 000,1 500 000,серый,...,False,Москва,Tiggo,1,1,0,0,Other,0,270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3010,Geely Monjaro (Xingyue L) I Рестайлинг,Geely,Monjaro (Xingyue L) I Рестайлинг,10,2.0,238,2025,3 830 000,3 830 000,серебристый,...,False,Москва,Monjaro,1,1,0,0,Crossover,1,319
3011,Geely Icon,Geely,Icon,1,1.5,181,2024,2 699 999,2 699 999,голубой,...,False,Москва,Icon,1,0,0,0,Other,0,80
3012,Geely Icon,Geely,Icon,10,1.5,181,2024,2 699 999,2 699 999,серый,...,False,Москва,Icon,1,0,0,0,Other,0,80
3013,Geely Atlas Pro,Geely,Atlas Pro,58 000,1.5,177,2022,2 100 000,2 100 000,синий,...,False,Московская область,Atlas,1,0,1,0,SUV,0,132


In [20]:
data.rename(columns={'Название': 'name',
                     'Модель': 'model',
                     'Марка': 'brand',
                     'Пробег': 'mileage',
                     'Мощность_1': 'power_1',
                     'Мощность_2': 'power_2',
                     'Год выпуска': 'year',
                     'Привод': 'drive',
                     'Цена': 'price',
                     'Цвет': 'color',
                     'Коробка_передач': 'gearbox',
                     'Тип_топлива': 'fuel_type',
                     'Город': 'city',
                     'Цвет_группа': 'color_group',
                     'Город_группа': 'city_group'}, inplace=True
                     )

In [21]:
data

,name,brand,model,mileage,power_1,power_2,year,price,"Цена до, руб",color,...,is_new,city_group,base_model,generation,is_restyling,is_pro,is_max,car_class,is_premium,model_popularity
4,Haval H3,Haval,H3,12 000,1.5,177,2024,2 320 000,2 320 000,серый,...,False,Москва,H3,1,0,0,0,SUV,0,4
32,Chery Tiggo 4 Pro I,Chery,Tiggo 4 Pro I,52 460,1.5,113,2024,1 490 000,1 490 000,серый,...,False,Москва,Tiggo,1,0,1,0,Other,0,270
35,Haval H3,Haval,H3,6 308,1.5,177,2024,2 650 000,2 650 000,чёрный,...,False,Москва,H3,1,0,0,0,SUV,0,4
57,Geely Okavango I Рестайлинг,Geely,Okavango I Рестайлинг,17 169,2.0,200,2023,2 980 000,2 980 000,синий,...,False,Московская область,Okavango,1,1,0,0,Other,0,7
60,Chery Tiggo 8 I Рестайлинг,Chery,Tiggo 8 I Рестайлинг,22 000,1.5,147,2022,1 500 000,1 500 000,серый,...,False,Москва,Tiggo,1,1,0,0,Other,0,270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3010,Geely Monjaro (Xingyue L) I Рестайлинг,Geely,Monjaro (Xingyue L) I Рестайлинг,10,2.0,238,2025,3 830 000,3 830 000,серебристый,...,False,Москва,Monjaro,1,1,0,0,Crossover,1,319
3011,Geely Icon,Geely,Icon,1,1.5,181,2024,2 699 999,2 699 999,голубой,...,False,Москва,Icon,1,0,0,0,Other,0,80
3012,Geely Icon,Geely,Icon,10,1.5,181,2024,2 699 999,2 699 999,серый,...,False,Москва,Icon,1,0,0,0,Other,0,80
3013,Geely Atlas Pro,Geely,Atlas Pro,58 000,1.5,177,2022,2 100 000,2 100 000,синий,...,False,Московская область,Atlas,1,0,1,0,SUV,0,132


In [22]:
data.year.value_counts()

year
2024    497
2025    251
2022    202
2023    195
2020     60
2021     52
2011     31
2018      7
2019      5
2009      3
2007      3
2014      3
2012      2
2010      1
2015      1
2013      1
2008      1
Name: count, dtype: int64

In [23]:
current_year = 2025
data['age'] = current_year - data['year'].astype(float)
data

,name,brand,model,mileage,power_1,power_2,year,price,"Цена до, руб",color,...,city_group,base_model,generation,is_restyling,is_pro,is_max,car_class,is_premium,model_popularity,age
4,Haval H3,Haval,H3,12 000,1.5,177,2024,2 320 000,2 320 000,серый,...,Москва,H3,1,0,0,0,SUV,0,4,1.0
32,Chery Tiggo 4 Pro I,Chery,Tiggo 4 Pro I,52 460,1.5,113,2024,1 490 000,1 490 000,серый,...,Москва,Tiggo,1,0,1,0,Other,0,270,1.0
35,Haval H3,Haval,H3,6 308,1.5,177,2024,2 650 000,2 650 000,чёрный,...,Москва,H3,1,0,0,0,SUV,0,4,1.0
57,Geely Okavango I Рестайлинг,Geely,Okavango I Рестайлинг,17 169,2.0,200,2023,2 980 000,2 980 000,синий,...,Московская область,Okavango,1,1,0,0,Other,0,7,2.0
60,Chery Tiggo 8 I Рестайлинг,Chery,Tiggo 8 I Рестайлинг,22 000,1.5,147,2022,1 500 000,1 500 000,серый,...,Москва,Tiggo,1,1,0,0,Other,0,270,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3010,Geely Monjaro (Xingyue L) I Рестайлинг,Geely,Monjaro (Xingyue L) I Рестайлинг,10,2.0,238,2025,3 830 000,3 830 000,серебристый,...,Москва,Monjaro,1,1,0,0,Crossover,1,319,0.0
3011,Geely Icon,Geely,Icon,1,1.5,181,2024,2 699 999,2 699 999,голубой,...,Москва,Icon,1,0,0,0,Other,0,80,1.0
3012,Geely Icon,Geely,Icon,10,1.5,181,2024,2 699 999,2 699 999,серый,...,Москва,Icon,1,0,0,0,Other,0,80,1.0
3013,Geely Atlas Pro,Geely,Atlas Pro,58 000,1.5,177,2022,2 100 000,2 100 000,синий,...,Московская область,Atlas,1,0,1,0,SUV,0,132,3.0


In [24]:
target = 'price'
numeric_feats = ['mileage', 'power_1', 'power_2', 'model_popularity', 'age', 'generation']
cat_feats = ['brand', 'color', 'color_group', 'gearbox', 'drive', 'fuel_type', 'city', 'city_group', 'base_model', 'car_class']
dummies = ['is_new', 'is_restyling', 'is_pro', 'is_max', 'is_premium']

In [25]:
for col in numeric_feats:
    try:
        data[col] = data[col].str.replace('\xa0', '').astype(float)
    except:
        data[col] = data[col].astype(float)
data                                     

,name,brand,model,mileage,power_1,power_2,year,price,"Цена до, руб",color,...,city_group,base_model,generation,is_restyling,is_pro,is_max,car_class,is_premium,model_popularity,age
4,Haval H3,Haval,H3,12000.0,1.5,177.0,2024,2 320 000,2 320 000,серый,...,Москва,H3,1.0,0,0,0,SUV,0,4.0,1.0
32,Chery Tiggo 4 Pro I,Chery,Tiggo 4 Pro I,52460.0,1.5,113.0,2024,1 490 000,1 490 000,серый,...,Москва,Tiggo,1.0,0,1,0,Other,0,270.0,1.0
35,Haval H3,Haval,H3,6308.0,1.5,177.0,2024,2 650 000,2 650 000,чёрный,...,Москва,H3,1.0,0,0,0,SUV,0,4.0,1.0
57,Geely Okavango I Рестайлинг,Geely,Okavango I Рестайлинг,17169.0,2.0,200.0,2023,2 980 000,2 980 000,синий,...,Московская область,Okavango,1.0,1,0,0,Other,0,7.0,2.0
60,Chery Tiggo 8 I Рестайлинг,Chery,Tiggo 8 I Рестайлинг,22000.0,1.5,147.0,2022,1 500 000,1 500 000,серый,...,Москва,Tiggo,1.0,1,0,0,Other,0,270.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3010,Geely Monjaro (Xingyue L) I Рестайлинг,Geely,Monjaro (Xingyue L) I Рестайлинг,10.0,2.0,238.0,2025,3 830 000,3 830 000,серебристый,...,Москва,Monjaro,1.0,1,0,0,Crossover,1,319.0,0.0
3011,Geely Icon,Geely,Icon,1.0,1.5,181.0,2024,2 699 999,2 699 999,голубой,...,Москва,Icon,1.0,0,0,0,Other,0,80.0,1.0
3012,Geely Icon,Geely,Icon,10.0,1.5,181.0,2024,2 699 999,2 699 999,серый,...,Москва,Icon,1.0,0,0,0,Other,0,80.0,1.0
3013,Geely Atlas Pro,Geely,Atlas Pro,58000.0,1.5,177.0,2022,2 100 000,2 100 000,синий,...,Московская область,Atlas,1.0,0,1,0,SUV,0,132.0,3.0


In [26]:
data['price'] = data['price'].str.replace('\xa0', '').astype(float)

In [27]:
cols = cat_feats + numeric_feats + dummies + [target]

data = data[cols].reset_index(drop=True)

In [28]:
data

,brand,color,color_group,gearbox,drive,fuel_type,city,city_group,base_model,car_class,...,power_2,model_popularity,age,generation,is_new,is_restyling,is_pro,is_max,is_premium,price
0,Haval,серый,Нейтральные,робот,полный,Бензин,Москва,Москва,H3,SUV,...,177.0,4.0,1.0,1.0,False,0,0,0,0,2320000.0
1,Chery,серый,Нейтральные,механика,передний,Бензин,Москва,Москва,Tiggo,Other,...,113.0,270.0,1.0,1.0,False,0,1,0,0,1490000.0
2,Haval,чёрный,Нейтральные,робот,полный,Бензин,Москва,Москва,H3,SUV,...,177.0,4.0,1.0,1.0,False,0,0,0,0,2650000.0
3,Geely,синий,Холодные,робот,передний,Бензин,Балашиха,Московская область,Okavango,Other,...,200.0,7.0,2.0,1.0,False,1,0,0,0,2980000.0
4,Chery,серый,Нейтральные,робот,передний,Бензин,Москва,Москва,Tiggo,Other,...,147.0,270.0,3.0,1.0,False,1,0,0,0,1500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,Geely,серебристый,Нейтральные,автомат,полный,Бензин,Москва,Москва,Monjaro,Crossover,...,238.0,319.0,0.0,1.0,False,1,0,0,1,3830000.0
1311,Geely,голубой,Холодные,робот,передний,Бензин,Москва,Москва,Icon,Other,...,181.0,80.0,1.0,1.0,False,0,0,0,0,2699999.0
1312,Geely,серый,Нейтральные,робот,передний,Бензин,Москва,Москва,Icon,Other,...,181.0,80.0,1.0,1.0,False,0,0,0,0,2699999.0
1313,Geely,синий,Холодные,робот,полный,Бензин,Одинцово,Московская область,Atlas,SUV,...,177.0,132.0,3.0,1.0,False,0,1,0,0,2100000.0


In [29]:
existing_columns = set(data.columns)
new_features = []

# Удельная мощность
if 'power_density' not in existing_columns:
        # Рассчитываем удельную мощность
        data['power_density'] = data["power_2"] / data["power_1"]
        new_features.append('power_density')
        print("Добавлен признак 'power_density'")

# Взаимодействие пробега и возраста
if 'mileage_age_interaction' not in existing_columns:
    mileage_col = 'Пробег, км' if 'Пробег, км' in existing_columns else 'mileage'

    if mileage_col in existing_columns and 'age' in data.columns:
        if data[mileage_col].dtype == 'object':
            mileage_numeric = data[mileage_col].copy()
            
            if 'is_new' in data.columns:
                if data['is_new'].dtype == bool:
                    mileage_numeric.loc[data['is_new'] == True] = '0'
                else:
                    mileage_numeric.loc[data['is_new'] == 1] = '0'
            
            mileage_numeric = mileage_numeric.replace({'Новый': '0'})
            
            mileage_numeric = pd.to_numeric(
                mileage_numeric.astype(str).str.replace(' ', ''), 
                errors='coerce'
            )
        else:
            mileage_numeric = data[mileage_col]
            
            if 'is_new' in data.columns:
                if data['is_new'].dtype == bool:
                    mileage_numeric.loc[data['is_new'] == True] = 0
                else:
                    mileage_numeric.loc[data['is_new'] == 1] = 0
        
        data['mileage_age_interaction'] = mileage_numeric * data['age']
        
        if 'is_new' in data.columns:
            if data['is_new'].dtype == bool:
                data.loc[data['is_new'] == True, 'mileage_age_interaction'] = 0
            else:
                data.loc[data['is_new'] == 1, 'mileage_age_interaction'] = 0
        
        new_features.append('mileage_age_interaction')
        print("Добавлен признак 'mileage_age_interaction'")



Добавлен признак 'power_density'
Добавлен признак 'mileage_age_interaction'


In [30]:
data.to_csv('cars_clean')